In [27]:
import tensorflow as tf
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt

In [28]:
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [29]:
dataset_dir = pathlib.Path(flowers_root)

In [30]:
class_names = np.array([item.name for item in dataset_dir.glob('*') if item.name!='LICENSE.txt'])

In [31]:
class_names

array(['tulips', 'daisy', 'roses', 'dandelion', 'sunflowers'],
      dtype='<U10')

In [32]:
dataset = tf.data.Dataset.list_files(str(dataset_dir/'*/*'))

In [33]:
validation_split = 0.2

In [34]:
DATASET_SIZE = len(list(dataset))
print("Dataset size: ", DATASET_SIZE)
train_size = int((1-validation_split) * DATASET_SIZE)
print("train size: ", train_size)
train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

Dataset size:  3670
train size:  2936


In [35]:
def augment_data(image):
    image = tf.image.resize_with_crop_or_pad(image, 180, 180) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[150, 150, 3]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

    return image

In [36]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  return parts[-2] == class_names

In [37]:
def load_img(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, 3, expand_animations=False)
    img = tf.cast(img, tf.float32)
    return img

In [38]:
def normalize(image):
    image = (image / 127.5) - 1
    return image

In [39]:
def resize(image,height, width):
    image = tf.image.resize(image, (height, width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

In [40]:
def load_image_with_label(image_path):
    label = get_label(image_path)
    img = load_img(image_path)
    return img, label

In [41]:
def load_image_train(image_file):
    image, label = load_image_with_label(image_file)
    image = augment_data(image)
    image = normalize(image)
    
    return image, label

In [42]:
def load_image_test(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image, 150, 150)
    image = normalize(image)

    return image, label

In [43]:
b_size = 32
s_size = 1000

In [44]:
train_dataset = train_dataset.map(load_image_train)
train_dataset = train_dataset.shuffle(s_size)
train_dataset = train_dataset.batch(b_size)

In [45]:
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(b_size)

In [46]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [47]:
base_model.trainable = False

In [48]:
num_classes = len(class_names)

In [49]:
model = tf.keras.Sequential([
                             base_model
])

In [50]:
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [52]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
history = model.fit(train_dataset, epochs=100, validation_data=validation_dataset)

Epoch 1/100
92/92 [==============================] - 49s 334ms/step - loss: 1.2449 - accuracy: 0.4765 - val_loss: 0.9568 - val_accuracy: 0.6172
Epoch 2/100
92/92 [==============================] - 31s 287ms/step - loss: 0.9605 - accuracy: 0.5947 - val_loss: 0.8103 - val_accuracy: 0.6812
Epoch 3/100
92/92 [==============================] - 31s 290ms/step - loss: 0.8883 - accuracy: 0.6366 - val_loss: 0.8955 - val_accuracy: 0.6403
Epoch 4/100
92/92 [==============================] - 31s 290ms/step - loss: 0.8613 - accuracy: 0.6339 - val_loss: 0.6742 - val_accuracy: 0.7520
Epoch 5/100
92/92 [==============================] - 31s 287ms/step - loss: 0.8371 - accuracy: 0.6649 - val_loss: 0.7066 - val_accuracy: 0.7262
Epoch 6/100
92/92 [==============================] - 31s 288ms/step - loss: 0.8117 - accuracy: 0.6676 - val_loss: 0.6978 - val_accuracy: 0.7439
Epoch 7/100
92/92 [==============================] - 31s 290ms/step - loss: 0.7927 - accuracy: 0.6809 - val_loss: 0.6018 - val_accuracy: